Encoders:
 - https://huggingface.co/monsoon-nlp/hindi-bert
 - https://huggingface.co/flax-community/roberta-hindi
 - https://huggingface.co/vasista22/whisper-hindi-small
 
Decoders:
- https://huggingface.co/csebuetnlp/mT5_m2o_hindi_crossSum
- https://huggingface.co/docs/transformers/model_doc/openai-gpt
- https://huggingface.co/docs/transformers/model_doc/ctrl
- https://huggingface.co/docs/transformers/model_doc/gpt2

In [82]:
# from transformers import EncoderDecoderModel, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/hindi-bert")
# model = EncoderDecoderModel.from_encoder_decoder_pretrained("monsoon-nlp/hindi-bert", "gpt2")

# model.config.decoder_start_token_id = tokenizer.cls_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

# # a hindi sentence
# input_ids = tokenizer(
#     'भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया है।',
#     return_tensors="pt",
# ).input_ids

# labels = tokenizer(
#     'राहुल गांधी ने कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया है।', return_tensors="pt",
# ).input_ids

# # the forward function automatically creates the correct decoder_input_ids
# outputs = model(input_ids=input_ids, labels=labels)

In [83]:
# outputs = model.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id)
# outputs

In [84]:
# tokenizer2 = AutoTokenizer.from_pretrained("gpt2")
# tokenizer2.decode(outputs[0], skip_special_tokens=True, predict_with_generate=True)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

test_df = pd.read_csv('/raid/home/dhruv/pankaji/NLP_PROJECT/Data/HindiNews_test.txt')
train_df = pd.read_csv('/raid/home/dhruv/pankaji/NLP_PROJECT/Data/hindi_train.txt')
print(train_df.shape, test_df.shape)

train_df = train_df.dropna()
test_df = test_df.dropna()

train_val_split = 0.15
train_df, val_df = train_test_split(train_df, test_size=train_val_split, random_state=42)
print(train_df.shape, val_df.shape, test_df.shape)

(21225, 4) (3000, 3)
(18041, 4) (3184, 4) (3000, 3)


In [2]:
train_df.head()

,Id,Heading,Summary,Article
8713,hindi_2023_train_8713,प्रोडक्शन के लिए एम्ब्रेयर और सुखोई की बातचीत ...,india looks partner sukhoi embraer to manufact...,भारत सरकार दूर-दराज इलाकों से बेहतर एयर कनेक्ट...
19043,hindi_2022_9186,UP: भदोही में नशे में धुत युवक ने महात्मा गांध...,जिसके बाद कांग्रेस कार्यकर्ता एकत्रित होकर गां...,भदोहीः उत्तर प्रदेश के भदोही में बीती शनिवार र...
17257,hindi_2022_7400,भारत के इस क्षेत्र में नहीं है Coronavirus का ...,भारत में कोविड-19 के मामले 97.35 लाख के पार चल...,कोच्चि: भारत में कोविड-19 के मामले 97.35 लाख क...
3156,hindi_2023_train_3156,12वीं में पासिंग परसेंटेज के आधार पर परीक्षा म...,NEET 2024 - No Minimum Marks Needed For NEET. ...,12वीं के बाद मेडिकल की पढ़ाई करने वाले छात्रों ...
15583,hindi_2022_5726,"उत्तर प्रदेश में कोरोना वायरस से 17 और मौतें, ...",उत्‍तर प्रदेश में शुक्रवार को कोरोना वायरस संक...,लखनऊ: उत्‍तर प्रदेश में शुक्रवार को कोरोना वाय...


In [3]:
import torch
from transformers import EncoderDecoderModel, AutoTokenizer, AutoModelForCausalLM,AutoModelForSeq2SeqLM, MT5ForConditionalGeneration

tokenizer_enc = AutoTokenizer.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum") #"aashay96/indic-gpt"
#model = EncoderDecoderModel.from_encoder_decoder_pretrained("ai4bharat/IndicBART","facebook/mbart-large-en-ro" )
tokenizer_dec = AutoTokenizer.from_pretrained("facebook/mbart-large-50")


#decoder_model.enc_to_dec_proj=torch.nn.Linear(768, len(tokenizer))
#decoder_model.enc_to_dec_proj.name = "lm_head"
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

encoder_model =  AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum").get_encoder()
#model = EncoderDecoderModel.from_encoder_decoder_pretrained(encoder_model=encoder_model, decoder_pretrained_model_name_or_path="facebook/mbart-large-50")
model_decoder = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50").get_decoder()
model_decoder.config.decoder_start_token_id = tokenizer_dec.pad_token_id#tokenizer.cls_token_id
encoder_model.config.pad_token_id = tokenizer_enc.pad_token_id
tokenizer_enc.add_tokens(["<sum>"], special_tokens=True )
tokenizer_enc.add_tokens(["<sum>"], special_tokens=True )
encoder_model = encoder_model.to(device)
model_decoder = model_decoder.to(device)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/raid/home/dhruv/miniconda3/envs/myenv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [41]:
AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50")

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [ ]:
model_decoder

MBartDecoder(
  (embed_tokens): Embedding(250054, 1024, padding_idx=1)
  (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
  (layers): ModuleList(
    (0-11): 12 x MBartDecoderLayer(
      (self_attn): MBartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): MBartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
  

In [54]:
encoder_model

MT5Stack(
  (embed_tokens): Embedding(250112, 768)
  (block): ModuleList(
    (0): MT5Block(
      (layer): ModuleList(
        (0): MT5LayerSelfAttention(
          (SelfAttention): MT5Attention(
            (q): Linear(in_features=768, out_features=768, bias=False)
            (k): Linear(in_features=768, out_features=768, bias=False)
            (v): Linear(in_features=768, out_features=768, bias=False)
            (o): Linear(in_features=768, out_features=768, bias=False)
            (relative_attention_bias): Embedding(32, 12)
          )
          (layer_norm): MT5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): MT5LayerFF(
          (DenseReluDense): MT5DenseGatedActDense(
            (wi_0): Linear(in_features=768, out_features=2048, bias=False)
            (wi_1): Linear(in_features=768, out_features=2048, bias=False)
            (wo): Linear(in_features=2048, out_features=768, bias=False)
            (dropout): Dropout(p=0.1, inplace=False

In [20]:
# train = iter(train_df['Heading'].iloc[i]+train_df['Article'].iloc[i] for i in range(len(train_df)))
# new_tokenizer = tokenizer.train_new_from_iterator(train, vocab_size=10000)
# new_tokens = set(new_tokenizer.vocab) - set(tokenizer.vocab)
# len(new_tokenizer.vocab)
# tokenizer.add_tokens(list(new_tokens))

In [4]:
model_decoder.resize_token_embeddings(len(tokenizer_dec))
#model.resize_token_embeddings(len(tokenizer))
encoder_model.resize_token_embeddings(len(tokenizer_enc))


Embedding(250101, 768)

In [5]:
import torch

In [10]:
import torch.nn as nn
import torch.optim as optim



In [42]:
class m(nn.Module):
    def __init__(self):
        super(m,self).__init__()
        self.device = device
        self.linear = nn.Linear(768, 1024, device = self.device)
        self.encoder_model = encoder_model
        self.decoder_model = model_decoder
        self.loss = nn.CrossEntropyLoss()
        self.lm_head = nn.Linear(1024,len(tokenizer_dec), device = self.device)
    def forward(self, input_ids, attention_mask, labels_input_ids, labels_attention_mask):
        out_enc = self.encoder_model(input_ids=input_ids,attention_mask=attention_mask)
        enc_hidden_state = out_enc.last_hidden_state
        enc_hidden_state = self.linear(enc_hidden_state)
        out_dec = self.decoder_model(input_ids = labels_input_ids, attention_mask = labels_attention_mask, encoder_hidden_states=enc_hidden_state) 
        logits = self.lm_head(out_dec.last_hidden_state)
        loss = self.loss(logits.squeeze(), labels_input_ids.squeeze())
        return logits ,loss           

In [43]:
mod = m()
optimizer = optim.Adam(mod.parameters(), lr=1e-3)
num_epochs = 3

In [45]:
def train(model, tokenizer_enc, tokenizer_dec, train_df, optimizer):
    model.train()
    total_loss = 0
    batch = 0
    batch_loss = 0
    batch_size = 1000
    for i, row in train_df.iterrows():
      
        input = tokenizer_enc("<sum>"+row["Article"], return_tensors="pt",max_length=1024, truncation=True)
        input_ids = input.input_ids.to(device)
        attention_mask = input.attention_mask.to(device)
        labels = tokenizer_dec(row['Summary'], return_tensors="pt")
        labels_input_ids = labels.input_ids.to(device)
        labels_attention_mask = labels.attention_mask.to(device)
        
        logits, loss = model(input_ids=input_ids, attention_mask=attention_mask,  labels_input_ids=labels_input_ids,labels_attention_mask=labels_attention_mask)
        #outputs = model(input_ids=input_ids, attention_mask=attention_mask,  labels=labels_input_ids, decoder_attention_mask=labels_attention_mask) 
        #loss = outputs.loss
        total_loss += loss.item()
        batch_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        batch += 1
        if batch % batch_size == 0:
            print(f"Batch {batch} Loss: {batch_loss / batch_size}")
            batch_loss = 0
    print(f"Training Loss: {total_loss / len(train_df)}")
        

def evaluate(model, tokenizer_enc, tokenizer_dec, val_df):
    model.eval()
    total_loss = 0
    for i, row in val_df.iterrows():
        input = tokenizer_enc("<sum>"+row["Article"], return_tensors="pt", max_length=1024, truncation=True)
        input_ids = input.input_ids.to(device)
        attention_mask = input.attention_mask.to(device)
        labels = tokenizer_dec(row['Summary'], return_tensors="pt")
        labels_input_ids = labels.input_ids.to(device)
        labels_attention_mask = labels.attention_mask.to(device)
        #outputs = model(input_ids=input_ids, attention_mask=attention_mask,  labels=labels_input_ids, decoder_attention_mask=labels_attention_mask)
        logits, loss = model(input_ids=input_ids, attention_mask=attention_mask,  labels_input_ids=labels_input_ids,labels_attention_mask=labels_attention_mask)
        total_loss += loss.item()
    print(f"Validation Loss: {total_loss / len(val_df)}")


for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    train(mod, tokenizer_enc, tokenizer_dec, train_df, optimizer)
    evaluate(mod, tokenizer_enc, tokenizer_dec, val_df)

Epoch 1
Batch 1000 Loss: 1.5505152671933173
Batch 2000 Loss: 1.136932954502292
Batch 3000 Loss: 1.2455086765419692
Batch 4000 Loss: 0.9942877303548158
Batch 5000 Loss: 0.9375281589161605
Batch 6000 Loss: 0.7536206559808925
Batch 7000 Loss: 0.71439530245075
Batch 8000 Loss: 0.6416664533203003
Batch 9000 Loss: 0.6225625697569922
Batch 10000 Loss: 0.543062920810422
Batch 11000 Loss: 0.5361014707612339
Batch 12000 Loss: 0.5252809669609997
Batch 13000 Loss: 0.5133743769332068
Batch 14000 Loss: 0.4544009827639093
Batch 15000 Loss: 0.4437603746164823
Batch 16000 Loss: 0.41798241723170215
Batch 17000 Loss: 0.3919906807463267
Batch 18000 Loss: 0.41697318682889456
Training Loss: 0.7128633632289189


TypeError: evaluate() missing 1 required positional argument: 'val_df'

In [46]:
head_text = test_df['Heading'][18]
text = test_df["Article"][19]
print(test_df['id'][6])
print(text)

input_ids = tokenizer_enc("sum>"+text, return_tensors="pt", truncation=True).input_ids.to(device)
print(input_ids.shape)

outputs = mod.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id, num_beams=4)
print(outputs)
# tokenizer2
summary = tokenizer_dec.decode(outputs[0], skip_special_tokens=True, predict_with_generate=True)
print("Summary:", summary)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


HindiNews_test_6
राष्ट्रपति द्रौपदी मुर्मू गुरुवार को असम जाएंगीं। वे 6 और 7 अप्रैल को इस राज्य के दौरे पर रहेंगीं। इस दौरान राष्ट्रपति यहां काजीरंगा नेशनल पार्क में एलिफेंट महोत्सव में भाग लेंगीं। मीडिया रिपोर्ट्स के मुताबिक राष्ट्रपति की यात्रा के दौरान 6 और 7 अप्रैल को काजीरंगा राष्ट्रीय उद्यान की दो रेंज पर्यटकों के लिए बंद रहेंगी।
हाथी प्रोजेक्ट के 30 साल पूरे होने का जश्न मनाया जा रहा
यह उत्सव राष्ट्रीय उद्यान में हर साल आयोजित किया जाता है। इस साल केंद्र और राज्य सरकार के पर्यावरण और वन विभाग संयुक्त रूप से इसका आयोजन कर रहे हैं। यह महोत्सव 7 और 8 अप्रैल को होगा। महोत्सव का आयोजन हाथी प्रोजेक्ट के 30 साल पूरे होने का जश्न मनाने और संरक्षण प्रयासों को बढ़ावा देने के लिए किया जा रहा है।
पर्यटकों के लिए बंद रहेगी जीप और हाथी सफारी
कार्यक्रम के दौरान पर्यटकों के लिए उद्यान में जीप और हाथी सफारी दो दिनों तक बंद रहेगी। काजीरंगा नेशनल पार्क असम के गोलाघाट और नागांव क्षेत्र में स्थित है। यह असम का सबसे पुराना उद्यान है जो उत्तर में ब्रह्मपुत्र नदी के किनारे और दक्षिण में कार्बी आंगलोंग 

AttributeError: 'm' object has no attribute 'generate'

In [29]:
test_df["Article"].iloc[3]
test_df["Heading"].iloc[3]
test_df.columns

Index(['id', 'Article', 'Heading'], dtype='object')